In [6]:
## Importing the required libraries ##
import numpy as np
import pandas as pd
from sklearn.gaussian_process import kernels
#from continuum_gaussian_bandits import ContinuumArmedBandit, GPR
from catboost import CatBoostClassifier, CatBoostRegressor
from sklearn.metrics import mean_squared_error
from sklearn.gaussian_process import kernels
from sklearn.gaussian_process import GaussianProcessRegressor

import dill 
import argparse
import datetime

from google.api_core import retry
from google.cloud import bigquery
from google.cloud import firestore
from google.cloud import pubsub_v1
from google.cloud import storage

In [4]:
class ContextualContinuumArmedBandit:
    """
    This class implements a contextual version of the ContinuumArmedBandit, which extends it to work
    in the contextual setting. We do this by augmenting the structure of the bandit to 
    incoroporate a contextual feature matrix. This identifies a unique context in which to train for 
    action/rewards. Thus, we train a ContinuumArmedBandit for each unique context encountered and use this
    as our model structure. Model API follows closely that defined by scikit-learn and also that by ContinuumArmedBandit in that 
    it merely extends that class to work for the contextual setting by specifying a context ID whenever calling similarly named methods such as fit or predict.
    
    """
        
    def __init__(self, contexts, oracle, bid_max_value=None, context_ids = None, convergence_rate=1.0, exploration_threshold=0.1, action_col_name='avg_cpc', kpi_name = 'avg_ctr'):
        self.context_dict = {} # keep stored all the trained bandit models to be called for prediction later on.
        
        if type(contexts) != pd.core.frame.DataFrame:
            print("WARNING: The contexts provided are not in the form of a Pandas DataFrame object. Attempting to convert, but no guarantees... ")
            try:
                contexts = pd.DataFrame(contexts)
            except:
                print("ERROR: Could not convert contexts into a Pandas DataFrame. Please transform it on the user end and pass as required...")
                raise
                
        if contexts.shape[0] != contexts.drop_duplicates().shape[0]:
            print("ERROR: The contexts provided are not unique. Please provide only unique lists of context information, i.e. a context matrix, to instantiate a trained ContextualContinuumArmedBandit. Returning...")
            raise
       
        if not bid_max_value:
            bid_max_value = 2.0
        
        # do not require a default list of context IDs. If none provided use a default index from 0 to N, the number of unique contexts
        if not context_ids:
            self.context_ids = np.arange(contexts.shape[0])
        else:
            self.context_ids = context_ids
        self.contexts = contexts.copy(deep=True) # keep a dictionary or dataframe containing all the contexts we wish to keep track of
        self.bid_max_value = bid_max_value # keep track of the max allowed bid value
        self.action_col_name = action_col_name # keep track of the name of action column name, by default avg_cpc
        self.kpi_name = kpi_name # keep track of the kpi name for optimization/i.e. to use as reward, default ctr
        self.exploration_threshold = exploration_threshold
        X = np.linspace(0, self.bid_max_value, 100)
        # setting X to be a 2D matrix, one data point per dimension, such that the kernel treats each point separately

        X_pred = np.atleast_2d(X).T
        self.num_contexts = len(contexts)
        # loop over the context IDs we
        for context_id in self.context_ids:
            print("INFO: Fitting continuum bandit for context: " + str(context_id + 1))
            df_context = pd.DataFrame(self.contexts.iloc[context_id, :]).transpose()
            df_context = df_context.loc[df_context.index.repeat(len(X))].reset_index(drop=True)
            df_context = pd.concat([df_context, pd.DataFrame({'avg_cpc': X})], axis = 1)
            
            #if not type(oracle) == google.cloud.aiplatform.models.Endpoint:
            y_pred = oracle.predict(df_context)
            #else:
            #    y_preds = oracle.predict(instances=[df_context.to_json()])
            # setting y to be a 2D matrix to match the dimensions of X when calculating the alpha parameter
            y_pred = np.atleast_2d(y_pred).T
            self.context_dict[self.context_ids[context_id]] = (ContinuumArmedBandit(X_pred, y_pred , convergence_rate=1.0), None, None)


    def select_action(self, context_id):
        """
        Select the next action to sample from. Can be thought of as similar to an acquisition function in Bayesian optimization
        
        Note
        ----
        context_id should preferably come from the same list of IDs as the contexts from the training data. It is very easy otherwise to predict for an incorrect context!
        
        Parameters
        ----------
        context_id : int
            This is the ID of the context in our context matrix (represented via a Pandas DataFrame object) which we want to generate a prediction for
            
        Returns
        -------
        result : float
            The result is the numeric action which will act as our next continuous action to take for this instance.
            
        """
        continuum_bandit = self.context_dict[context_id][0] # get the bandit trained algorithm for this context id
        x = continuum_bandit.select_action() # query this bandits select_action() method to retrieve the next query point based on its acquisition function.
        return x
        
    def get_x_best(self, X, context_id):
        """
        Selects the next action to take based on the acquisition function defined for the continuum gaussian bandit        

        Parameters
        ----------
        X : array(n_samples, ) 
            An array of points for which to calculate a merit score via the acquisition function of the bandit model
            thus giving rise to candidate points to choose for the next query point x_best
            
        context_id : int type
            The specific context id for which to retrieve the next best action to follow

        Returns
        -------
        x_best : float
            The next best point to try sample at as the next action
        """
        x_best = self.context_dict[context_id][0].get_x_best(X) # query the get_x_best function of the ContinuumArmedBandit associated with this context id
        
        return x_best


    def update(self, df_contexts, actions, rewards, start_from_checkpoint=True):
        """
        Updates the contextual bandit given newly observed rewards for the actions set by the model.
        
        Parameters
        ----------
        df_contexts : array(n_contexts, n_features) or pd.DataFrame of shape (n_contexts, n_features)
            Matrix of covariates for the available data.
        actions : array(n_samples, ), float type
            Arms or actions that were chosen for each observations.
        rewards : array(n_samples, ), [0,1]
            Rewards that were observed for the chosen actions. Must be binary rewards 0/1.
        start_from_checkpoint : bool
            If the policy was previously fit to data, 
            it will use previously trained models along with adding this new data to old data. In this case,
            will only refit the models that have new data according to actions and rewards specified.
        
        Returns
        -------
        self : obj
            No return type
        """
        # important checks to run, check of the object is of type dataframe
        if type(df_contexts) != pd.core.frame.DataFrame:
            print("WARNING: The contexts provided are not in the form of a Pandas DataFrame object. Attempting to convert, but no guarantees... ")
            try:
                df_contexts = pd.DataFrame(df_contexts)
            except:
                print("ERROR: Could not convert contexts into a Pandas DataFrame. Please transform it on the user end and pass as required...")
                raise
                
        # check if the data in the context matrix passed match what has been trained on historically
        if not self.contexts.columns.isin(df_contexts.columns).all():
            print("ERROR: Columns provided in df_contexts does not correspond to the column trained on for this model. Please provide a data frame matching the following column signature: " + str(self.contexts.columns))
            raise 
            
        # check the reverse holds in case we have a different set in some slight way
        if not df_contexts.columns.isin(self.contexts.columns).all():
            print("ERROR: Columns provided in df_contexts does not correspond to the column trained on for this model. Please provide a data frame matching the following column signature: " + str(self.contexts.columns))
            raise 
            
        # for each unique context, find the similar or corresponding context in our data and run this through the bandit model
        # for this context, updating the actions and rewards following this.
        for context_id in range(df_contexts.drop_duplicates().shape[0]):
            try:
                context = df_contexts.iloc[context_id, :]
                
                context_location = np.where((self.contexts == context).all(axis=1))[0][0] # getting the first possible index from the np.where results which are of the form np.array(np.array, dtype)
                continuum_bandit = self.context_dict[context_location][0]
                X = actions[context_id]
                y = rewards[context_id]
                continuum_bandit.update(X, y)
                
            except:
                print("ERROR: Unsampled or unseen context, please update or re-fit the model with the context included.")
                raise
        
        
    def fit(self, num_rounds):
        """
        Continues to fit the model from the last trained point in terms of parameters. In essence, 
        
        1) it samples a random context to train, 
        2) samples a next action point to query,
        3) receives a reward from the oracle environment simulation model,
        4) updates the bandit trained for this context       

        Parameters
        ----------
        num_rounds : int type 
            The number of iterations or samples which will be carried out for training of this model.
            
        Returns
        -------

        """
        np.random.seed(42)
        for round_num in num_rounds:
            sample_context_id = np.random.randint(self.num_contexts)
            sampled_context = self.contexts[sample_context_id]
            continuum_bandit = self.context_dict[sample_context_id][0]
            
            epsilon = np.random.random() # pick a random number between 0 and 1
            
            # if the number is greater than the model exploration threshold
            # pick a next query point based on select_action or using the model's acquisition function
            # or just exploit the next best point in terms of known best strategy
            if epsilon < self.exploration_threshold:
                x = continuum_bandit.predict()
            else:
                x = continuum_bandit.select_action()
            
            y_pred = self.oracle.predict(sampled_context.append(x))
            continuum_bandit.update(x, y_pred)
            
    
    def predict(self, new_contexts):
        """
        Using the currently trained-on parameters of the model, generate a series of predictions for the new contexts
        provided to the model. This follows the following steps,
        
        1) it attemps to match currently known contexts to those provided,
        2) with probability epsilon, retrieve a next action query point based on the model's acquisition function
        3) otherwise do greedy-style exploitation of best known point
        4) return the models best decision given the above.

        Parameters
        ----------
        new_contexts : pd.DataFrame of shape (n_contexts, n_features)
            The context feature matrix defining the unique contexts we would like to query for a prediction.
            
        Returns
        -------
        df_results : pd.DataFrame of shape (n_contexts, n_features + 1)
            This returns the above new_contexts dataframe object but we append a column to it with the next actions to be taken.
            
        """
        if type(new_contexts) != pd.core.frame.DataFrame:
            print("WARNING: The contexts provided are not in the form of a Pandas DataFrame object. Attempting to convert, but no guarantees... ")
            try:
                new_contexts = pd.DataFrame(new_contexts)
            except:
                print("ERROR: Could not convert contexts into a Pandas DataFrame. Please transform it on the user end and pass as required...")
                raise
                
                
        result_set = []
        for context_id in range(new_contexts.drop_duplicates().shape[0]):
            try:
                context = self.contexts.iloc[context_id, :]
                
                context_location = np.where((self.contexts == context).all(axis=1))[0][0] # getting the first possible index from the np.where results which are of the form np.array(np.array, dtype)
                continuum_bandit = self.context_dict[context_location][0] # getting the trained bandit model from the context_dict
                
                epsilon = 1.0 # random number, such that with prob. 0.1 we choose a random action to explore via select_action, and otherwise predict as usual
            
                if epsilon < self.exploration_threshold:
                    x = continuum_bandit.select_action()
                else:
                    x = continuum_bandit.predict()
            except:
                print("ERROR: Unsampled or unseen context, please update or re-fit the model with the context included.")
                raise 
            result_set.append(x[0][0])
            
        df_results = pd.DataFrame(new_contexts.copy(deep=True)) # create deepcopy
        df_results['avg_cpc'] = result_set
        return df_results
    
    def partial_fit(self, contexts_new, X_new, y_new, reward_new):
        """ TODO - for online bandit algorithm """
        for context in contexts_new:
            try:
                sampled_context = self.context_dict[context]
            except:
                print("ERROR: Unsampled or unseen context, please update or re-fit the model with the context included.")
                raise


In [5]:
class ContinuumArmedBandit:
    """
    This class implements ContinuumArmedBandit, which is a multi-armed bandit extended
    to work in the continuous action space definition of the problem. The method of achieving this is 
    by keeping track of the distribution of rewards and expected rewards for each point in the action space.
    In the usual MAB setting this boils down to keeping track of summary statistics for each action and using algorithms like
    LinUCB, Exp3 etc. to sample from the space itself.
    
    In our case, we do something similar in that we use the methodology of LinUCB and use the upper confidence interval predicted
    for the rewards around a given action in order to pick the next best action to sample, however, 
    we approximate the probabilities differently in this continuous setting: We use 
    Gaussian processes to approximate the distribution of rewards varying across our continuous action space!
    
    This allows us to use simple to use and easy to extend methods such as GaussianProcessRegressor from scikit-learn (see docs for more).
    
    We merely extend this class in the GPR implementation which accompanies this class, in order to implement this acquisition function approach in 
    order to sample our probability space for the next best action.
    
    """
    def __init__(self, X, y, convergence_rate=1.0):
        """
        Intialize the class with the necessary X and y values, in this case representing the actions and rewards to train on, respectively.
        The convergence rate determines by what factor to update the kernel and Gaussian parameters W, K, alpha and gamma
        
        Parameters
        ----------
        X : array(n_samples, n_actions) or pd.DataFrame of shape (n_samples, n_actions)
            The actions to train on. Usually n_actions is 1, meaning that shape (n_samples, ) results for both input types
        y : array(n_samples, ) or pd.DataFrame or pd.Series of shape (n_samples, 1)
            The rewards which result from the corresponding actions in X above.
            
        Returns
        -------
            
        """
        self.X = X.copy()
        self.y = y.copy() # set X and y as local parameters
        self.gpr = GPR(self.X, self.y, convergence_rate=convergence_rate) # define our custom defined GaussianProcessRegressor model
        self.N = self.X.shape[0]
        self.convergence_rate = convergence_rate
        self.alpha = self.calc_alpha(self.gpr.K, self.gpr.noise_var, self.y) # update the alpha values associated with reward or y-values
        self.gamma = self.calc_gamma(self.gpr.K, self.gpr.noise_var, self.X) # update the gamma values associated with actions or data points or x-values

    def select_action(self):
        """
        Select the next action to sample from. Can be thought of as similar to an acquisition function in Bayesian optimization

        Parameters
        ----------

        Returns
        -------
        x : float
            The result is the numeric action which will act as our next continuous action to take for this instance.
            
        """
        x = self.get_x_best(self.X)
        return x
    
    def update(self, X, y):
        """
        Stacks the new X and y values onto the existing ones, updating the parameters of the model and hence refitting it totally on the new appended data.        

        Parameters
        ----------
        X : array(n_samples, ) 
            An array of points which correspond to the action points to update the model with
            
        y : array(n_samples)
            An array containing the associated reward values to model against X

        Returns
        -------

        """
        self.X = np.vstack((self.X, X)) # append copies of this dataset in the locally stored class values
        self.y = np.vstack((self.y, y)) 
        self.gpr = GPR(self.X, self.y, convergence_rate=self.convergence_rate)
        self.N = self.X.shape[0] # update dataset size convenience parameter


        self.alpha = self.calc_alpha(self.gpr.K, self.gpr.noise_var, self.y) # update alpha for this new dataset and post-retraining of the GPR model
        self.gamma = self.calc_gamma(self.gpr.K, self.gpr.noise_var, self.X) # update gamma for this new dataset and post-retraining of the GPR model-retraining of the GPR model

    def get_x_best(self, X):
        """
        Selects the next action to take based on the acquisition function defined for the continuum gaussian bandit        

        Parameters
        ----------
        X : array(n_samples, ) 
            An array of points for which to calculate a merit score via the acquisition function of the bandit model
            thus giving rise to candidate points to choose for the next query point x_best
            
        Returns
        -------
        x_best : float
            The next best point to try sample at as the next action
        """
        
        merit_best = 0
        x_best = None
        for j in range(self.N):
            x = X[j]
            s = self.get_s(x, X)
            x = x + s
            x_merit = self.get_merit(x, X)
            if x_merit > merit_best:
                x_best = x
                merit_best = x_merit
        return x_best

    def get_s(self, x, X):
        """
        Returns the mean prediction for the value x relative to the action space X, with confidence envelope s. 
        Works in the derivative space to approximate gradients of the model fit       

        Parameters
        ----------
        x : float type
            the action query point to evaluate the mean and standard deviation derivatives for following from the kernel function
        X : array(n_samples, ) 
            An array of points which correspond to the action points to update the model with
            
        Returns
        -------
        s : float
            The derivative mean prediction for point x relative to X with derivative envelope 
        """
        der_mean = self.get_derivative_mean(x, X)
        der_std = self.get_derivative_std(x, X)
        s = der_mean + der_std
        return s
    
    def get_derivative_std(self, x, X):
        """
        Returns the derivative standard deviation prediction for the point x relative to action space data X w.r.t. the kernel       

        Parameters
        ----------
        x : float type
            the action query point to evaluate the mean and standard deviation derivatives for following from the kernel function
        X : array(n_samples, ) 
            An array of points which correspond to the action points to update the model with
            
        Returns
        -------
        s : float
            The derivative standard deviation envelope for point x relative to X w.r.t. the kernel
        """
        der_std = 0
        W = self.gpr.W
        k_prime = self.gpr.k
        std = self.get_std(x, X)
        for i in range(self.N):
            for j in range(self.N):
                der_std += (2.0 / std) * self.gamma[i,j] * W.dot((X[i] + X[j]) / 2.0 - x) * k_prime(x, (X[i] + X[j]) / 2.0) 
        return der_std


    def get_derivative_mean(self, x, X):
        """
        Retrieves the model kernel for the dataset X provided. TODO: update this method to simply use self.X. 
        At the moment uses a user provided X, which in this case is self.X passed internally among the class methods.
        

        Parameters
        ----------
        X : array(n_samples, ) 
            An array of points which correspond to the action points to update the model with
            
        Returns
        -------
        kernel : sklearn.gaussian_process.kernels types
            A kernel class which consists of the kernel matrix with normalized length scales along each dimension of X, determined by X.shape[1] of course. 
            it is a combination between a radial basis function kernel and some white noise
        """
        der_mean = 0
        W = self.gpr.W
        k = self.gpr.k
        for i in range(self.N):
            der_mean += W.dot(X[i] - x) * k(x, X[i]) * self.alpha[i]
        return der_mean


    def get_merit(self, x, X):
        """
        Retrieves the model kernel for the dataset X provided. TODO: update this method to simply use self.X. 
        At the moment uses a user provided X, which in this case is self.X passed internally among the class methods.
        

        Parameters
        ----------
        X : array(n_samples, ) 
            An array of points which correspond to the action points to update the model with
            
        Returns
        -------
        kernel : sklearn.gaussian_process.kernels types
            A kernel class which consists of the kernel matrix with normalized length scales along each dimension of X, determined by X.shape[1] of course. 
            it is a combination between a radial basis function kernel and some white noise
        """
        mean = self.get_mean(x, X)
        var = self.get_std(x, X)
        merit = mean + var
        return merit
    
    def predict(self):
        """
        Retrieves the model kernel for the dataset X provided. TODO: update this method to simply use self.X. 
        At the moment uses a user provided X, which in this case is self.X passed internally among the class methods.
        

        Parameters
        ----------
        X : array(n_samples, ) 
            An array of points which correspond to the action points to update the model with
            
        Returns
        -------
        kernel : sklearn.gaussian_process.kernels types
            A kernel class which consists of the kernel matrix with normalized length scales along each dimension of X, determined by X.shape[1] of course. 
            it is a combination between a radial basis function kernel and some white noise
        """
        test_x = np.linspace(0.01, np.max(self.X))
        
        val_pred, val_var = self.predict_reward(np.atleast_2d(test_x).T)
        
        
        action_idx = np.where(val_pred + val_var == np.max(val_pred + val_var))
        action = self.X[action_idx]
        
        return action
        
    
    def predict_reward(self, x_arr):
        """
        Retrieves the model kernel for the dataset X provided. TODO: update this method to simply use self.X. 
        At the moment uses a user provided X, which in this case is self.X passed internally among the class methods.
        

        Parameters
        ----------
        X : array(n_samples, ) 
            An array of points which correspond to the action points to update the model with
            
        Returns
        -------
        kernel : sklearn.gaussian_process.kernels types
            A kernel class which consists of the kernel matrix with normalized length scales along each dimension of X, determined by X.shape[1] of course. 
            it is a combination between a radial basis function kernel and some white noise
        """
        val_pred = []
        val_var = []
        for x in x_arr:
            mean = self.get_mean(x, self.X)
            var = self.get_std(x, self.X)
            val_pred.append(mean)
            val_var.append(var)
        return val_pred, val_var

    def get_mean(self, x, X):
        """
        Retrieves the model kernel for the dataset X provided. TODO: update this method to simply use self.X. 
        At the moment uses a user provided X, which in this case is self.X passed internally among the class methods.
        

        Parameters
        ----------
        X : array(n_samples, ) 
            An array of points which correspond to the action points to update the model with
            
        Returns
        -------
        kernel : sklearn.gaussian_process.kernels types
            A kernel class which consists of the kernel matrix with normalized length scales along each dimension of X, determined by X.shape[1] of course. 
            it is a combination between a radial basis function kernel and some white noise
        """
        mean = 0
        k = self.gpr.k
        for j in range(self.N):
            mean += k(x,X[j]) * self.alpha[j]
        return mean

    def get_std(self, x, X):
        """
        Retrieves the model kernel for the dataset X provided. TODO: update this method to simply use self.X. 
        At the moment uses a user provided X, which in this case is self.X passed internally among the class methods.
        

        Parameters
        ----------
        X : array(n_samples, ) 
            An array of points which correspond to the action points to update the model with
            
        Returns
        -------
        kernel : sklearn.gaussian_process.kernels types
            A kernel class which consists of the kernel matrix with normalized length scales along each dimension of X, determined by X.shape[1] of course. 
            it is a combination between a radial basis function kernel and some white noise
        """
        
        k = self.gpr.k
        k_prime = self.gpr.k_prime
        kxx = k(x,x)
        a = 0
        for i in range(self.N):
            for j in range(self.N):
                a += k_prime(x, 0.5 * (X[i] + X[j])) * self.gamma[i,j]
        var = np.sqrt(kxx - a)
        return var
        
    def calc_alpha(self, K, noise_var, y):
        """
        Retrieves the model kernel for the dataset X provided. TODO: update this method to simply use self.X. 
        At the moment uses a user provided X, which in this case is self.X passed internally among the class methods.
        

        Parameters
        ----------
        X : array(n_samples, ) 
            An array of points which correspond to the action points to update the model with
            
        Returns
        -------
        kernel : sklearn.gaussian_process.kernels types
            A kernel class which consists of the kernel matrix with normalized length scales along each dimension of X, determined by X.shape[1] of course. 
            it is a combination between a radial basis function kernel and some white noise
        """
        alpha = np.linalg.inv((K + noise_var * np.eye(self.N))).dot(y)
        return alpha
    
    def calc_gamma(self, K, noise_var, X):
        """
        Retrieves the kernel matrix for the dataset X provided. TODO: update this method to simply use self.X. 
        At the moment uses a user provided X, which in this case is self.X passed internally among the class methods.
        

        Parameters
        ----------
        X : array(n_samples, ) 
            An array of points which correspond to the action points to update the model with
            
        Returns
        -------
        kernel : array(n_actions, n_actions) type, n_action = X.shape[0]
            A kernel matrix with shape the first dimension of X, determined by X.shape[0] of course. 
            it is a matrix which applies the kernel function of choice to each point in the data
        """
        beta = self.get_Beta(X)
        gamma = np.multiply(np.linalg.inv((K + noise_var * np.eye(self.N))), beta)
        return gamma
    
    def get_Beta(self, X):
        """
        Retrieves the Beta matrix for the dataset X provided. TODO: update this method to simply use self.X. 
        At the moment uses a user provided X, which in this case is self.X passed internally among the class methods.
        

        Parameters
        ----------
        X : array(n_samples, ) 
            An array of points which correspond to the action points to update the model with
            
        Returns
        -------
        Beta : array(n_actions, n_actions) type, n_action = X.shape[0]
            A beta matrix with shape the first dimension of X, determined by X.shape[0] of course. 
            it is a matrix which applies the beta function of choice to each point in the data
        """
        N = X.shape[0]
        Beta = np.zeros((N,N))
        for i in range(N):
            for j in range(N):
                Beta[i,j] = self.beta(X[i], X[j])
        return Beta
    
    def beta(self, x1, x2):
        """
        Retrieves the beta coefficient between x1 and x2, which is an unscaled kernel function between them used in calculating self.beta

        Parameters
        ----------
        x1 : float 
            A specific point to calculate beta_x1x2  
        x2 : float 
            A specific point to calculate beta_x1x2
            
        Returns
        -------
        beta_x1x2 : float
            The average  beta for x1, x2
        """
        W = self.gpr.W
        beta_x1x2 = np.exp(-0.25 * (x1 - x2).T.dot(W).dot(x1 - x2))
        return beta_x1x2

    def get_q_mean(self, x, X):
        """
        Calculates the mean q value or q parameter of x given data X, using the second kernel function derivative.
        
        This can be interpreted as the average change in the slope relative to other data points
        
        Parameters
        ----------
        x : float
            A specific point to calculate q at relative to X
        X : array(n_samples, ) 
            An array of points which correspond to the action points to update the model with
            
        Returns
        -------
        q_mean : float
            The average value of q for x given X
        """
        q_mean = 0
        kpp = self.gpr.k_prime_prime
        for i in range(self.N):
            q_mean += np.absolute(kpp(x, X[i])) * self.alpha[i]
        return q_mean

class GPR(GaussianProcessRegressor):
    def __init__(self, X, y, convergence_rate=1.0):
        """
        Intialize the class with the necessary X and y values, in this case representing the actions and rewards to train on, respectively.
        The convergence rate determines by what factor to update the kernel and Gaussian parameters W, K, alpha and gamma
        
        Parameters
        ----------
        X : array(n_samples, n_actions) or pd.DataFrame of shape (n_samples, n_actions)
            The actions to train on. Usually n_actions is 1, meaning that shape (n_samples, ) results for both input types
        y : array(n_samples, ) or pd.DataFrame or pd.Series of shape (n_samples, 1)
            The rewards which result from the corresponding actions in X above.
            
        Returns
        -------
            
        """
        
        self.X = X # set the data set and initialize some class variables
        self.y = y
        self.W = None
        self.K = None
        self.noise_var = None
        self.converge_rate_sq = np.power(convergence_rate, 2) # set the square of the convergence for the super model
        GaussianProcessRegressor.__init__(self, kernel=self.get_kernel(self.X)) # get the kernel corresponding to this dataset
        self.fit(self.X, self.y) # fit the model using the super class fit method
        self.update_W() # update the W matrix with length scale parameters
        self.update_K(self.X) # update the kernel matrix using the kernel and the dataset
        self.update_noise_var() # update the noise variable using the super class noise variable definition
	
    def update(self, X, y):
        """
        Stacks the new X and y values onto the existing ones, updating the parameters of the model and hence refitting it totally on the new appended data.        

        Parameters
        ----------
        X : array(n_samples, ) 
            An array of points which correspond to the action points to update the model with
            
        y : array(n_samples)
            An array containing the associated reward values to model against X

        Returns
        -------

        """
        
        self.X = np.vstack((self.X, X.copy()))
        self.y = np.append(self.y, y.copy())
        self.fit(self.X, self.y)
        self.update_W()
        self.update_K(self.X)
        self.update_noise_var()
    
    def get_kernel(self, X):
        """
        Retrieves the model kernel for the dataset X provided. TODO: update this method to simply use self.X. 
        At the moment uses a user provided X, which in this case is self.X passed internally among the class methods.
        
        Notes
        -----
        
        TODO: Adjust the kernel to be problem specific. This is very hard to do generally, as it requires domain specific knowledge in ranges of values
        your features will take. See here for more explanation. 
        
        SEE HERE: https://github.com/scikit-learn/scikit-learn/issues/7563
        
        Parameters
        ----------
        X : array(n_samples, ) 
            An array of points which correspond to the action points to update the model with
            
        Returns
        -------
        kernel : sklearn.gaussian_process.kernels types
            A kernel class which consists of the kernel matrix with normalized length scales along each dimension of X, determined by X.shape[1] of course. 
            it is a combination between a radial basis function kernel and some white noise
        """
        length_scale = np.random.normal(loc=1.0,scale=.1,size=X.shape[1])
        rbf = kernels.RBF(length_scale=length_scale)
        # we set these particular levels of noise to ensure better convergence of the GPR (tested in practice to work better)
        wk = kernels.WhiteKernel(noise_level=1e-5, noise_level_bounds=(1e-10, 1e-4))
        kernel = rbf + wk
        return kernel

    def update_K(self, X):
        """
        Retrieves the kernel matrix for the dataset X provided. TODO: update this method to simply use self.X. 
        At the moment uses a user provided X, which in this case is self.X passed internally among the class methods.
        

        Parameters
        ----------
        X : array(n_samples, ) 
            An array of points which correspond to the action points to update the model with
            
        Returns
        -------
        kernel : array(n_actions, n_actions) type, n_action = X.shape[0]
            A kernel matrix with shape the first dimension of X, determined by X.shape[0] of course. 
            it is a matrix which applies the kernel function of choice to each point in the data
        """
        N = X.shape[0]
        K = np.zeros((N,N)) # generate empty N x N matrix of zeroes to instantiate
        for i in range(N):
            for j in range(N):
                K[i,j] = self.k(X[i], X[j]) # apply kernel function of choice defined below to each pair of data points
        self.K = K # set the kernel in the class variables

    def update_W(self):
        """
        Retrieves the kernel matrix for the dataset X provided. TODO: update this method to simply use self.X. 
        At the moment uses a user provided X, which in this case is self.X passed internally among the class methods.
        

        Parameters
        ----------
        X : array(n_samples, ) 
            An array of points which correspond to the action points to update the model with
            
        Returns
        -------
        kernel : sklearn.gaussian_process.kernels types
            A kernel class which consists of the kernel matrix with normalized length scales along each dimension of X, determined by X.shape[1] of course. 
            it is a combination between a radial basis function kernel and some white noise
        """
        length_scales = self.kernel_.get_params()['k1__length_scale']
        if len(self.X.shape) == 1 or self.X.shape[1] == 1:
            length_scales = np.array([length_scales]) # length scales is a singular number, because X is a one dimensional dataset, hence we make it into an array
            
        w = 1.0 / np.power(length_scales, 2)
        W = np.diag(w)
        self.W = W

    def update_noise_var(self):
        """
        Updates the noise variable in the GaussianProcessRegressor model and internally based on updated values
        of self.X and self.y

        Parameters
        ----------

        Returns
        -------
        """
        noise_var = self.kernel_.get_params()['k2__noise_level']
        self.noise_var = noise_var


    def k(self, x1, x2):
        """
        Kernel function defined as variant of matern kernel. Takes in two numeric vectors, x1 and x2 from
        the data self.X and applies the kernel to them. Returns kernel value to be used in 
        constructing a kernel matrix

        Parameters
        ----------
        x1 : float or array(1, n_dim) if multi-dimensional.
            The first data point to be applied against the kernel with x2
        x2 : float or array(1, n_dim) if multi-dimensional.
            The second data point to be applied against the kernel with x1

        Returns
        -------
        k_x2x2 : float type
            The return value which is the result of the kernel function applied to x1 and x2
            
        """
        k_x1x2 = self.converge_rate_sq * np.exp(-0.5 * (x1 - x2).T.dot(self.W).dot(x1 - x2))
        return k_x1x2

    def k_prime(self, x1, x2):
        """
        Kernel first derivative of the usual kernel function defined in self.k
        This is a helper function for calculating gradients, increases, decreases etc.
        When arriving at query points via the acquisition functions


        Parameters
        ----------
        x1 : float or array(1, n_dim) if multi-dimensional.
            The first data point to be applied against the kernel with x2
        x2 : float or array(1, n_dim) if multi-dimensional.
            The second data point to be applied against the kernel with x1

        Returns
        -------
        k_x2x2 : float type
            The return value which is the result of the kernel function applied to x1 and x2
            
        """
        k_prime_x1x2 = self.converge_rate_sq * np.exp(-1.0 * (x1 - x2).T.dot(self.W).dot(x1 - x2))
        return k_prime_x1x2
             
    def k_prime_prime(self, x1, x2):
        """
        Kernel second derivative of the usual kernel function defined in self.k
        This is a helper function for calculating gradients, increases, decreases etc.
        When arriving at query points via the acquisition functions

        Parameters
        ----------
        x1 : float or array(1, n_dim) if multi-dimensional.
            The first data point to be applied against the kernel with x2
        x2 : float or array(1, n_dim) if multi-dimensional.
            The second data point to be applied against the kernel with x1

        Returns
        -------
        k_x2x2 : float type
            The return value which is the result of the kernel function applied to x1 and x2
            
        """
        k_prime_prime_x1x2 = self.converge_rate_sq * np.exp(-1.0/6.0 * (x1 - x2).T.dot(self.W).dot(x1 - x2))
        return k_prime_prime_x1x2


In [7]:
project_id = 'aa-cmab-product'

In [12]:
%%bigquery df_ad_group_summary --project $project_id
select * from `aa-cmab-product.RawData.ProductionProcessedData`

Downloading: 100%|██████████| 32757/32757 [00:01<00:00, 29665.69rows/s]


In [19]:
def train_bigquery_table(df):
    """
    Example usage of models, oracle models etc.training them using a locally downloaded CSV file in the form of the JOT Google Ads
    file data schema (see ./docs/data_docs/). We preprocess the raw docs in the form that it is done to put it into the BigQuery ProductionProcessedData table
    
    Trains a catboost oracle model and then trains a bandit using this model.

    Additional fitting can be done via the fit_rounds, set to 0, or add more.
    
    Parameters
    ----------
    dataset_name : str
        Example file of the form JOT Google Ads export
        
    Returns
    -------

    """
    #dataset_name='C:\\Users\\PrasunGhosh\\Desktop\\OPERATIONAL_TESTING_DataSet-Final-Amplify_202108_20210823_QID10507419_20210824_125858_0.txt'
    #df = pd.read_csv('C:\\Users\\PrasunGhosh\\Desktop\\OPERATIONAL_TESTING_DataSet-Final-Amplify_202108_20210823_QID10507419_20210824_125858_0.txt', delimiter='\t')
    #df.columns = ['client_id',
    #              'campaign_id',
    #              'group_id',
    #              'account_descriptive_name',
    #              'ad_network_type',
    #              'avg_position',
    #              'campaign_name',
    #              'city_criteria_id',
    #              'clicks',
    #              'cost',
    #              'impressions',
    #              'country_criteria_id',
    #              'date',
    #              'device',
    #              'external_customer_id',
    #              'metro_criteria_id',
    #              'region_criteria_id']

    #df['date'] = pd.to_datetime(df['date'], format='%Y-%m-%d')
    #df['week_day'] = df['date'].dt.day_name()
    #df['weekday_num'] = df['date'].dt.week
    #
    #id_fields = ['client_id', 'campaign_id', 'group_id', 'city_criteria_id',
    #       'country_criteria_id', 'external_customer_id', 'metro_criteria_id',
    #       'region_criteria_id']
    #
    #df[id_fields] = df[id_fields].astype(object)
    
    group_by_fields = ['client_id',
    'external_customer_id',
    'group_id', 
    'account_descriptive_name', 
    'ad_network_type',
    'campaign_id', 
    'campaign_name',  
    'week_day',
    'weekday_num']
    
    
    #df_grouped = df.groupby(group_by_fields).agg({'clicks': ['mean', 'sum'], 'impressions': 'sum', 'cost': ['mean', 'sum']}).reset_index()
    #df_grouped.columns = ["_".join(a) for a in df_grouped.columns.to_flat_index()]
    #df_grouped.columns=df_grouped.columns.str.strip('_')
    #df_grouped['cost_sum'] = df_grouped['cost_sum']/1000000
    #df_grouped['cost_sum'] = df_grouped['cost_sum']/1000000
    #df_grouped['avg_ctr'] = df_grouped['clicks_sum']/df_grouped['impressions_sum']
    #df_grouped['avg_cpc'] = df_grouped['cost_sum']/df_grouped['clicks_sum']
    #df_grouped['total_cost'] = df_grouped['cost_sum']
    #df_grouped['total_clicks'] = df_grouped['clicks_sum']
    #df_grouped['total_impressions'] = df_grouped['impressions_sum']
    #df_grouped['avg_clicks'] = df_grouped['clicks_mean']
    #df_grouped['avg_cost'] = df_grouped['cost_mean']
    
    df_grouped=df
    
    df_grouped = df_grouped.drop(['weekday_num','clicks_sum', 'impressions_sum', 'cost_sum', 'cost_mean', 'clicks_mean'], axis=1, errors='ignore')
    df_grouped = df_grouped.fillna(0)
    df_grouped = df_grouped[df_grouped.avg_cpc > 0]
    
    df_train = df_grouped.drop(['avg_cpc', 'avg_ctr'], axis=1)
    
    group_by_fields.remove('weekday_num')
    print(df_grouped.columns)
    
    group_by_fields = ['client_id',
    'external_customer_id',
    'group_id', 
    'account_descriptive_name', 
    'ad_network_type',
    'campaign_id', 
    'campaign_name',  
    'week_day']

    train_data = df_grouped[group_by_fields + ['avg_cpc']]
    
    train_labels = df_grouped['avg_ctr']
    
    df_train=df_train[group_by_fields]

    
    cat_features = [i for i in range(len(group_by_fields))]
    model = CatBoostRegressor(iterations=1000)
    model.fit(train_data,
              train_labels,
              cat_features,
              verbose=True)
    
    
    train_preds = model.predict(train_data)
    
    
    print('RMSE error of catboost oracle: ' + str(np.sqrt(mean_squared_error(train_labels, train_preds))))
    print('Median of target variable: ' + str(np.median(train_labels)))
    print('Mean of target variable: ' + str(np.mean(train_labels)))
    # train the bandit example
    df_train=df_train.drop_duplicates().reset_index()
    bandit = ContextualContinuumArmedBandit(df_train, model, bid_max_value=1.0)
    
    dill.dump(bandit, open("training data/ContextualContinuumArmedBanditCloud_JOT.dill", "wb"))

In [20]:
train_bigquery_table(df_ad_group_summary)

Index(['client_id', 'external_customer_id', 'group_id',
       'account_descriptive_name', 'ad_network_type', 'campaign_id',
       'campaign_name', 'week_day', 'total_clicks', 'total_cost', 'avg_clicks',
       'avg_cost', 'total_impressions', 'avg_cpc', 'avg_ctr'],
      dtype='object')
Learning rate set to 0.064091
0:	learn: 0.0641194	total: 10.1ms	remaining: 10.1s
1:	learn: 0.0631663	total: 13.8ms	remaining: 6.88s
2:	learn: 0.0622386	total: 19.2ms	remaining: 6.39s
3:	learn: 0.0614054	total: 28.1ms	remaining: 6.99s
4:	learn: 0.0606090	total: 35ms	remaining: 6.97s
5:	learn: 0.0599306	total: 40.5ms	remaining: 6.71s
6:	learn: 0.0592829	total: 46.7ms	remaining: 6.63s
7:	learn: 0.0587108	total: 51.9ms	remaining: 6.43s
8:	learn: 0.0581476	total: 57.8ms	remaining: 6.37s
9:	learn: 0.0576551	total: 63ms	remaining: 6.24s
10:	learn: 0.0572302	total: 68.3ms	remaining: 6.14s
11:	learn: 0.0567399	total: 75ms	remaining: 6.17s
12:	learn: 0.0563244	total: 81.1ms	remaining: 6.16s
13:	learn: 0.0559283

CatBoostError: Bad value for num_feature[non_default_doc_idx=0,feature_idx=8]="Sunday": Cannot convert 'b'Sunday'' to float

In [ ]:
def train_local(dataset_name):
    """
    Example usage of models, oracle models etc.training them using a locally downloaded CSV file in the form of the JOT Google Ads
    file data schema (see ./docs/data_docs/). We preprocess the raw docs in the form that it is done to put it into the BigQuery ProductionProcessedData table
    
    Trains a catboost oracle model and then trains a bandit using this model.

    Additional fitting can be done via the fit_rounds, set to 0, or add more.
    
    Parameters
    ----------
    dataset_name : str
        Example file of the form JOT Google Ads export
        
    Returns
    -------

    """
    #dataset_name='C:\\Users\\PrasunGhosh\\Desktop\\OPERATIONAL_TESTING_DataSet-Final-Amplify_202108_20210823_QID10507419_20210824_125858_0.txt'
    df = pd.read_csv('C:\\Users\\PrasunGhosh\\Desktop\\OPERATIONAL_TESTING_DataSet-Final-Amplify_202108_20210823_QID10507419_20210824_125858_0.txt', delimiter='\t')
    df.columns = ['client_id',
                  'campaign_id',
                  'group_id',
                  'account_descriptive_name',
                  'ad_network_type',
                  'avg_position',
                  'campaign_name',
                  'city_criteria_id',
                  'clicks',
                  'cost',
                  'impressions',
                  'country_criteria_id',
                  'date',
                  'device',
                  'external_customer_id',
                  'metro_criteria_id',
                  'region_criteria_id']

    df['date'] = pd.to_datetime(df['date'], format='%Y-%m-%d')
    df['week_day'] = df['date'].dt.day_name()
    df['weekday_num'] = df['date'].dt.week
    
    id_fields = ['client_id', 'campaign_id', 'group_id', 'city_criteria_id',
           'country_criteria_id', 'external_customer_id', 'metro_criteria_id',
           'region_criteria_id']
    
    df[id_fields] = df[id_fields].astype(object)
    
    group_by_fields = ['client_id',
    'external_customer_id',
    'group_id', 
    'account_descriptive_name', 
    'ad_network_type',
    'campaign_id', 
    'campaign_name',  
    'week_day',
    'weekday_num']
    
    
    df_grouped = df.groupby(group_by_fields).agg({'clicks': ['mean', 'sum'], 'impressions': 'sum', 'cost': ['mean', 'sum']}).reset_index()
    df_grouped.columns = ["_".join(a) for a in df_grouped.columns.to_flat_index()]
    df_grouped.columns=df_grouped.columns.str.strip('_')
    df_grouped['cost_sum'] = df_grouped['cost_sum']/1000000
    df_grouped['cost_sum'] = df_grouped['cost_sum']/1000000
    df_grouped['avg_ctr'] = df_grouped['clicks_sum']/df_grouped['impressions_sum']
    df_grouped['avg_cpc'] = df_grouped['cost_sum']/df_grouped['clicks_sum']
    df_grouped['total_cost'] = df_grouped['cost_sum']
    df_grouped['total_clicks'] = df_grouped['clicks_sum']
    df_grouped['total_impressions'] = df_grouped['impressions_sum']
    df_grouped['avg_clicks'] = df_grouped['clicks_mean']
    df_grouped['avg_cost'] = df_grouped['cost_mean']
    
    df_grouped = df_grouped.drop(['weekday_num','clicks_sum', 'impressions_sum', 'cost_sum', 'cost_mean', 'clicks_mean'], axis=1, errors='ignore')
    df_grouped = df_grouped.fillna(0)
    df_grouped = df_grouped[df_grouped.avg_cpc > 0]
    
    df_train = df_grouped.drop(['avg_cpc', 'avg_ctr'], axis=1)
    
    group_by_fields.remove('weekday_num')
    print(df_grouped.columns)
    
    train_data = df_grouped[group_by_fields + ['avg_cpc']]
    
    train_labels = df_grouped['avg_ctr']
    
    cat_features = [i for i in range(len(group_by_fields))]
    model = CatBoostRegressor(iterations=1000)
    model.fit(train_data,
              train_labels,
              cat_features,
              verbose=True)
    
    train_preds = model.predict(train_data)
    print('RMSE error of catboost oracle: ' + str(np.sqrt(mean_squared_error(train_labels, train_preds))))
    print('Median of target variable: ' + str(np.median(train_labels)))
    print('Mean of target variable: ' + str(np.mean(train_labels)))
    # train the bandit example
    bandit = ContextualContinuumArmedBandit(df_train, model, bid_max_value=1.0)
    
    dill.dump(bandit, open("C:\\Users\\PrasunGhosh\\Desktop\\ContextualContinuumArmedBanditCloud_JOT_v1.dill", "wb"))
